**Pre-processing given dataset**




Need to upload history.csv and test.csv to run this notebook

In [ ]:
import pandas as pd
import numpy as np
import sklearn as sk
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import RandomOverSampler

Let's load the data!

In [ ]:
df = pd.read_csv("history.csv")
df.info()

FileNotFoundError: [Errno 2] No such file or directory: 'history.csv'

Let's display some data!

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
df.head(20)

Consider categorical data using one-hot encoding.



In [ ]:
numeric_columns = []
non_numeric_columns = []
for columns in df.columns:
    if df[columns].dtype in ["float64", "int64", "int8"]:
        numeric_columns.append(columns)
    else:
        non_numeric_columns.append(columns)

for column in non_numeric_columns:
    df[column] = df[column].astype('category')
    categories = df[column].cat.categories
    codes = df[column].cat.codes
    cat_to_code = {}
    code_to_cat = {}
    for i in range(len(categories)):
        cat = categories[i]
        cat_to_code[cat] = i
        code_to_cat[i] = cat
    df[column] = codes

df.head(20)

Consider missing data by assuming the most likely option.

In [ ]:
data = df.copy()


for columnb in df.columns:
    if len(df[columnb].dropna()) == 0:
        most_common = -1
    else: most_common = df[columnb].dropna().mode()[0]
    data[columnb].fillna(value=most_common, inplace=True)

data.head(20)

Consider the imbalanced data by oversampling.

In [ ]:
oversampler = RandomOverSampler(sampling_strategy="minority")
oversampled_data, oversampled_target = oversampler.fit_resample(data, pd.Series(data["label"]))
oversampled_data.shape

oversampled_data.head(20)

Saving our new data!

In [ ]:
oversampled_data.to_csv("./processed_data.csv")

**LETS PROCESS TEST TOO!!!**

In [ ]:
df = pd.read_csv("test.csv")
df.info()

numeric_columns = []
non_numeric_columns = []
for columns in df.columns:
    if df[columns].dtype in ["float64", "int64", "int8"]:
        numeric_columns.append(columns)
    else:
        non_numeric_columns.append(columns)

for column in non_numeric_columns:
    df[column] = df[column].astype('category')
    categories = df[column].cat.categories
    codes = df[column].cat.codes
    cat_to_code = {}
    code_to_cat = {}
    for i in range(len(categories)):
        cat = categories[i]
        cat_to_code[cat] = i
        code_to_cat[i] = cat
    df[column] = codes

data = df.copy()

for columnb in df.columns:
    if len(df[columnb].dropna()) == 0:
        most_common = -1
    else: most_common = df[columnb].dropna().mode()[0]
    data[columnb].fillna(value=most_common, inplace=True)

data.to_csv("./processed_test.csv")

**CLASSICAL ML**

Analyze customer behavior within the wealth management services domain. The dataset, consisting of many columns, aggregates information from various tables, offering insights into customer interactions, financial transactions, trading activities, and overall financial behaviors. The focus is on predicting customer churn and estimating lifetime value.

In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import confusion_matrix, classification_report, ConfusionMatrixDisplay, accuracy_score, recall_score, precision_score, f1_score
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier, RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
import xgboost as xgb
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import f1_score

In [ ]:
df = pd.read_csv("processed_data.csv")

In [ ]:
df

In [ ]:
filtered_df = df[df['label'] == 1]
filtered_df

In [ ]:
X = df.drop(["label"], axis=1)
y = df["label"]

In [ ]:
from sklearn.model_selection import train_test_split
#train test split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)

Try different classifying algorithms

In [ ]:
classifier = DecisionTreeClassifier()
classifier_cv_scores = cross_val_score(estimator=classifier, X=X_train, y=y_train, cv=5)
print(f"Average Decision Tree CV Score: {sum(classifier_cv_scores)/len(classifier_cv_scores)}")

classifier.fit(X_train, y_train) # decision tree
y_pred = classifier.predict(X_val)
f1 = f1_score(y_val, y_pred)
print(f"F1 Score: {f1}")

In [ ]:
clf = AdaBoostClassifier(n_estimators=100, learning_rate = 1, random_state=42)
clf.fit(X_train, y_train) # Adaboost
y_pred = clf.predict(X_val)
f1 = f1_score(y_val, y_pred)
print(f"F1 Score: {f1}")

In [ ]:
gbc = GradientBoostingClassifier(n_estimators = 100, learning_rate = 1, max_depth = None)
gbc.fit(X_train, y_train) # Gradient Boost
y_pred = gbc.predict(X_val)
f1 = f1_score(y_val, y_pred)
print(f"F1 Score: {f1}")

In [ ]:
knn = KNeighborsClassifier(n_neighbors = 8)
knn_cv_scores = cross_val_score(estimator=knn, X=X_train, y=y_train, cv=5)
print(f"Average KNN CV Score: {sum(knn_cv_scores)/len(knn_cv_scores)}")

knn.fit(X_train, y_train) # KNN
y_pred = knn.predict(X_val)
f1 = f1_score(y_val, y_pred)
print(f"F1 Score: {f1}")

In [ ]:
xgc = xgb.XGBClassifier(n_estimators=100, learning_rate = 1, booster = "gbtree", random_state = 42, max_depth = 13, max_samples = 0.9)
xgc.fit(X_train, y_train) # XGBoost
y_pred = xgc.predict(X_val)
f1 = f1_score(y_val, y_pred)
print(f"F1 Score: {f1}")

In [ ]:
logreg_model = LogisticRegression(random_state=42)
logreg_model.fit(X_train, y_train) # logistic regression
y_pred = logreg_model.predict(X_val)
f1 = f1_score(y_val, y_pred)
print(f"F1 Score: {f1}")

In [ ]:
svc = SVC(kernel = "rbf", class_weight = "balanced", gamma = "scale")
svc.fit(X_train, y_train) # SVM
y_pred = svc.predict(X_val)
f1 = f1_score(y_val, y_pred)
print(f"F1 Score: {f1}")

In [ ]:
rfc = RandomForestClassifier(n_estimators=100)
rfc.fit(X_train, y_train) # random forest
y_pred = rfc.predict(X_val)
f1 = f1_score(y_val, y_pred)
print(f"F1 Score: {f1}")

In [ ]:
rfc = RandomForestClassifier(n_estimators=100)
rfc.fit(X_train, y_train) # random forest

In [ ]:
feature_importances = rfc.feature_importances_
print("Feature Importances:", feature_importances)

In [ ]:
# Assuming feature_importances is your array of feature importances
top_indices = np.argsort(feature_importances)[::-1][:5]  # Get the indices of top 5 features

# Print the top 5 features and their importances
print("Top 5 Features:")
for i, idx in enumerate(top_indices):
    print(f"Feature {i+1}: Index={idx}, Importance={feature_importances[idx]}")

In [ ]:
top_feature_columns = df.iloc[:, top_indices]
combined_df = pd.concat([top_feature_columns, df['label']], axis=1)

pd.set_option('display.max_rows', 500)
display(combined_df.head(100))

In [ ]:
grouped = combined_df.groupby('label')
median_label_0 = grouped.median().loc[0]
median_label_1 = grouped.median().loc[1]
print("Median for label 0:")
print(median_label_0)
print("\nMedian for label 1:")
print(median_label_1)

In [ ]:
rfc = RandomForestClassifier(n_estimators=100)
param_grid = {'n_estimators': [50, 100, 200, 300, 400, 500]}

grid = GridSearchCV(rfc, param_grid)
grid.fit(X_train, y_train)

**NEURAL NETWORK**

In [ ]:
import pandas as pd

In [ ]:
#import data
df = pd.read_csv("processed_data.csv")
pd.set_option('display.max_columns', None)

In [ ]:
#drop empty
df.dropna(how='all', axis=1, inplace=True)
display(df)

In [ ]:
#split to features and labels
X = df.drop(["label"], axis=1)
#drop id and unnamed
X = X.drop(["id"], axis=1)
X = X.drop(["Unnamed: 0"], axis = 1)
y = df["label"]
display(X)
display(y)

In [ ]:
from sklearn.model_selection import train_test_split

# Identify numeric columns
numeric_columns = [
    col for col in X.columns
    if X[col].dtype in ["float64", "int64"]
]
X = X[numeric_columns]

#train test split, 80% train
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
X_train

In [ ]:
#feature scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)
display(X_train)
display(X_test)

In [ ]:
import torch
import torch.utils.data
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
class ANN(nn.Module):
    def __init__(self, input_dim = 104, output_dim = 1):
        super(ANN, self).__init__()
        #optimize numbers of neurons and # of layers
        # Input Layer 104 (features) -> 64
        self.fc1 = nn.Linear(input_dim, 64)
        # 64 -> 64
        self.fc2 = nn.Linear(64, 64)
        # 64 -> 32
        self.fc3 = nn.Linear(64, 32)
        # 32 -> 32
        self.fc4 = nn.Linear(32, 16)
        # 32 -> output layer 1
        self.output_layer = nn.Linear(16,1)
        # Dropout 20% to reduce overfitting
        self.dropout = nn.Dropout(0.2)

    # Feed Forward Function
    def forward(self, x):
        # ReLU
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = self.output_layer(x)

        return nn.Sigmoid()(x)

In [ ]:
# Create NN model
model = ANN(input_dim = 104, output_dim = 1)

In [ ]:
import torch
import torch.utils.data
from torch.autograd import Variable

# convert to torch from numpy
X_train = torch.from_numpy(X_train)
y_train = torch.from_numpy(y_train.to_numpy()).view(-1,1)

X_test = torch.from_numpy(X_test)
y_test = torch.from_numpy(y_test.to_numpy()).view(-1,1)

In [ ]:
# Make torch datasets from train and test sets
train = torch.utils.data.TensorDataset(X_train,y_train)
test = torch.utils.data.TensorDataset(X_test,y_test)

# Create train and test data loaders
# choose batch: 32, 64. 128, 256
train_loader = torch.utils.data.DataLoader(train, batch_size = 128, shuffle = True)
test_loader = torch.utils.data.DataLoader(test, batch_size = 128, shuffle = True)


In [ ]:
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler

# Binary cross entropy loss that performs the best to predict either 0 or 1
loss_fn = nn.BCELoss()

# specify learning rate, experiment with it
lr = 0.0005
# experimentation with multiple optimizers
#optimizer = optim.SGD(model.parameters(), lr=0.01, weight_decay= 1e-6, momentum = 0.9,nesterov = True)
optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay= 1e-6)
#scheduler = lr_scheduler.LinearLR(optimizer, start_factor=1.0, end_factor=1.0, total_iters=20)


TRAIN AND VALIDATE

In [ ]:
# between 10-50
epochs = 10

epoch_list = []
train_loss_list = []
train_f1_list = []
val_loss_list = []
val_f1_list = []

model.train()

for epoch in range(epochs):
    train_loss = 0.0
    val_loss = 0.0

    # Calculate true positive, false positive, true negative, false negative
    tp = 0
    fp = 0
    tn = 0
    fn = 0

    # pass in train data
    for data,target in train_loader:
        data = Variable(data).float()
        target = Variable(target).type(torch.FloatTensor)
        # clear the gradients of all optimized variables
        optimizer.zero_grad()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        predicted = (torch.round(output))

        tp += torch.sum((predicted == 1) & (target == 1))
        fp += torch.sum((predicted == 1) & (target == 0))
        tn += torch.sum((predicted == 0) & (target == 0))
        fn += torch.sum((predicted == 0) & (target == 1))

        # calculate loss
        loss = loss_fn(output, target)
        # backward pass: compute gradient of the loss with respect to model parameters
        loss.backward()
        # perform a single optimization step (parameter update)
        optimizer.step()# update running training loss
        train_loss += loss.item()*data.size(0)

 # calculate average training loss over an epoch
    train_loss = train_loss/len(train_loader.dataset)

    precision = tp/(tp + fp)
    recall = tp/(tp+fn)
    #calculate f1 score for training
    f1= (2*precision*recall)/(precision+recall)
    #record for visualization
    train_f1_list.append(f1)
    train_loss_list.append(train_loss)
    print('Epoch: {} \tTraining Loss: {:.4f}'.format(
        epoch+1,
        train_loss,
        ))
    print('tp: {}  fp: {}  tn: {}  fn: {}  f1: {:.6f}'.format(
        tp,
        fp,
        tn,
        fn,
        f1
        ))

    #validate with test data
    with torch.no_grad():
      ttp = 0
      tfp = 0
      ttn = 0
      tfn = 0
      for data,target in test_loader:

        data = Variable(data).float()
        target = Variable(target).type(torch.FloatTensor)

        output = model(data)
        predicted = (torch.round(output))

        ttp += torch.sum((predicted == 1) & (target == 1))
        tfp += torch.sum((predicted == 1) & (target == 0))
        ttn += torch.sum((predicted == 0) & (target == 0))
        tfn += torch.sum((predicted == 0) & (target == 1))
        loss = loss_fn(output, target)
        val_loss += loss.item()*data.size(0)

      test_precision = ttp/(ttp + tfp)
      test_recall = ttp/(ttp+tfn)
      test_f1= (2*test_precision*test_recall)/(test_precision+test_recall)
      val_loss = val_loss/len(test_loader.dataset)
      val_loss_list.append(val_loss)
      val_f1_list.append(test_f1)
      print('test f1: {:.6f}'.format(
        test_f1
        ))
      print('Testing Loss: {:.4f}'.format(val_loss))

    # Move to next epoch
    #scheduler.step() to update lr
    epoch_list.append(epoch + 1)

Plot learning curves to see if it's overfitting with epoch

In [ ]:
import matplotlib.pyplot as plt

plt.plot(epoch_list,train_loss_list, label='training')
plt.plot(epoch_list, val_loss_list, label='validating')
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.title("Loss vs Number of Epochs")
plt.legend()
plt.show()


plt.plot(epoch_list,train_f1_list, label='training')
plt.plot(epoch_list, val_f1_list, label='validating')
plt.xlabel("Epochs")
plt.ylabel("F1 Score")
plt.title("F1 Scores vs Number of Epochs")
plt.legend()
plt.show()


Now let's run with test data

In [ ]:
test_df = pd.read_csv("processed_test.csv")
ids = test_df["id"]
test_df = test_df.drop(["id"], axis =1)
test_df = test_df.drop(["Unnamed: 0"], axis =1)
pd.set_option('display.max_columns', None)
display(test_df)
test_df = sc.fit_transform(test_df)
test_df = torch.from_numpy(test_df)
test_df = Variable(test_df).float()

In [ ]:
#to record the predictions
results = []
model.eval()

# run model with test data
with torch.no_grad():
    for data in test_df:
        output = model(data)
        pred = int((torch.round(output.data[0])).item())
        results.append(pred)

Turn results into into submission.csv

In [ ]:
import csv
filename = "submission.csv"
header = ["id", "predicted"]
ids = ids

def turn_to_churn(n):
  if 0 == n:
    return "no churn"
  if 1 == n:
    return "churn"
  return "error"

predicted = map(turn_to_churn, results)

rows = [[id, pv] for id, pv in zip(ids, predicted)]
# Include the header
before_csv = [header] + rows

In [ ]:
with open(filename, 'w', newline='') as file:
    writer = csv.writer(file)

    # Write each row of data to the CSV file
    writer.writerows(before_csv)